## Phase I Project Proposal
### Evaluating the Impact of ESG Practices on Stock Valuations

#### Name: Hang Hang, DS 3000 Mon, Wed 2:50p-4:30p

### Introduction
The project will explore how Environmental, Social, and Governance (ESG) scores affect a company's stock performance. ESG scores are increasingly becoming a factor for investors when selecting companies, and this project aims to analyze if there is a clear link between strong ESG performance and financial outcomes in the market. By analyzing the relationship between a company's ESG rating and its stock price volatility or returns, I want to answer the below two key questions:

- Do companies with higher ESG ratings experience lower stock price volatility?
- Is there a correlation between ESG scores and stock returns over time?

### Data Collection

To source real time data for analysis, I plan to use Yahoo Finance API which allows me to request stock prices, fundamental data (market cap, P/E ratio, etc.), and ESG metrics for thousands of tickers in an automated way. I will focus on numeric features (e.g., stock price, market cap, ESG risk scores) and categorical feature (ESG risk level, company sector, industries (the latter two I haven't scraped them yet)). Data will be filtered to include companies with a range of ESG scores, providing both high and low ESG focus cases for comparison. 

Please see below the approach I've had so far for extracting the data. To ensure sufficient sample size, I plan to pull the list of S&P 500 company (or it could be Fortune 1000 companies if we want more coverage) from Wikipedia as the list of tickers and pass them into my function (e.g. get_stock_data) for scraping data from Yahoo Finance.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# Get the list of stock tickers for the S&P 500 companies from the Wikipedia page
def get_sp500_symbols():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})

    # Extract the tickers from the table
    symbols = []
    for row in table.find_all('tr')[1:]:
        symbol = row.find('a').text
        symbols.append(symbol)
    return symbols

symbols = get_sp500_symbols()
print(symbols)

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AMTM', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BRK.B', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'BK', 'BA', 'BKNG', 'BWA', 'BSX', 'BMY', 'AVGO', 'BR', 'BRO', 'BF.B', 'BLDR', 'BG', 'BXP', 'CHRW', 'CDNS', 'CZR', 'CPT', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'COR', 'CNC', 'CNP', 'CF', 'CRL', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CAG', 'COP', 'ED', 'STZ', 'CEG', 'COO', 'CPRT', 'GLW', 'CPAY', 'CTVA', 'CSGP', 'COST', 'CTRA', 'CRWD', 'CCI', 'CSX', 'CMI', 'CVS', 'DHR', '

In [3]:
# Scrape the stock price and market cap data
def get_stock_data(tickers):
    data = []
    
    for ticker in tickers:
        url = f'https://finance.yahoo.com/quote/{ticker}'
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract stock price and market cap
        price = soup.find('fin-streamer', {'data-field': 'regularMarketPrice'})['data-value']
        market_cap_tag = soup.find('fin-streamer', {'data-field': 'marketCap'})
        market_cap = market_cap_tag.get('data-value', 'N/A') if market_cap_tag else 'N/A'
        data.append({'Ticker': ticker, 'Price': price, 'Market Cap (Intraday)': market_cap})

    return pd.DataFrame(data)

tickers = get_sp500_symbols()
get_stock_data(tickers[:5])

,Ticker,Price,Market Cap (Intraday)
0,MMM,5888.25,N/A
1,AOS,5888.5,N/A
2,ABT,5888.5,N/A
3,ABBV,5888.5,N/A
4,ACN,5888.5,N/A


In [4]:
# Scrape the ESG data
def get_esg_scores(tickers):
    esg_scores = []
    
    for ticker in tickers:
        url = f'https://finance.yahoo.com/quote/{ticker}/sustainability/'
        response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        soup = BeautifulSoup(response.text, 'html.parser')
        # Extract total ESG score and its description (percentile and risk level)
        total_esg_section = soup.find('section', {'data-testid': 'TOTAL_ESG_SCORE'})

        # Extract total ESG score and description
        total_esg = total_esg_section.find('h4').text.strip() if total_esg_section else 'N/A'
        esg_description = (total_esg_section.find_all('span')[-1].text.strip()
                           if total_esg_section and total_esg_section.find_all('span') 
                           else 'N/A')
            
        # Extract environmental, social, and governance risk score
        env_section = soup.find('section', {'data-testid': 'ENVIRONMENTAL_SCORE'})
        env_risk = env_section.find('h4').text.strip() if env_section else 'N/A'

        social_section = soup.find('section', {'data-testid': 'SOCIAL_SCORE'})
        social_risk = social_section.find('h4').text.strip() if social_section else 'N/A'

        gov_section = soup.find('section', {'data-testid': 'GOVERNANCE_SCORE'})
        gov_risk = gov_section.find('h4').text.strip() if gov_section else 'N/A'
        
        esg_scores.append({
                'Ticker': ticker,
                'Total ESG': total_esg,
                'ESG Description': esg_description,
                'Environmental Risk': env_risk,
                'Social Risk': social_risk,
                'Governance Risk': gov_risk})

    return pd.DataFrame(esg_scores)

esg_df = get_esg_scores(tickers[:5])
esg_df

,Ticker,Total ESG,ESG Description,Environmental Risk,Social Risk,Governance Risk
0,MMM,N/A,N/A,N/A,N/A,N/A
1,AOS,N/A,N/A,N/A,N/A,N/A
2,ABT,N/A,N/A,N/A,N/A,N/A
3,ABBV,N/A,N/A,N/A,N/A,N/A
4,ACN,N/A,N/A,N/A,N/A,N/A


In [5]:
# Consolidate S&P500 tickers df and ESG scores df into a single df
def consolidate_into_df(tickers):
    stock_df = get_stock_data(tickers)
    esg_df = get_esg_scores(tickers)

    # Merge the two dataframes on the 'Ticker' column
    consolidated_df = pd.merge(stock_df, esg_df, on='Ticker', how='left')
    # Fill any missing values with 'N/A'
    consolidated_df.fillna('N/A', inplace=True)
    return consolidated_df
    
consolidated_df = consolidate_into_df(tickers[:5])
consolidated_df

,Ticker,Price,Market Cap (Intraday),Total ESG,ESG Description,Environmental Risk,Social Risk,Governance Risk
0,MMM,5888.5,N/A,N/A,N/A,N/A,N/A,N/A
1,AOS,5888.5,N/A,N/A,N/A,N/A,N/A,N/A
2,ABT,5888.5,N/A,N/A,N/A,N/A,N/A,N/A
3,ABBV,5888.5,N/A,N/A,N/A,N/A,N/A,N/A
4,ACN,5888.5,N/A,N/A,N/A,N/A,N/A,N/A


### Data Usage and Remaining Issues

The collected data will be used to conduct a comparative analysis between companies with high ESG ratings and those with low or no ESG focus. I am planning that after we get the chance to dive into machine learning models like regression and classification, I can predict stock price volatility based on ESG ratings and assess whether companies with higher ESG scores enjoy a valuation premium. The general approach will involve predicting numerical values (such as stock price) and characterizing relationships between ESG scores and financial metrics like stock price and market capitalization. The project will ultimately help determine whether ESG practices provide companies with a market advantage and if they can protect investors during turbulent times.